In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import time, json

# 1. 啟動 Chrome
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # 使用完整版視窗
driver = webdriver.Chrome(service=Service(), options=chrome_options)

# 2. 打開 IOTA DAG 視覺化頁面
driver.get("https://explorer.iota.org/mainnet/visualizer")
time.sleep(10)  # 給前端動畫 10 秒初始化時間

# 3. 點擊「暫停」按鈕（右上角）
try:
    pause_button = driver.find_element(By.XPATH, '//button[contains(@class, "pause")]')
    pause_button.click()
    print("✅ 已點擊暫停鍵")
except Exception as e:
    print("❌ 無法點擊暫停鍵：", e)

# 4. 等待 DAG 結構變數 `__graph` 出現，最多等 60 秒
print("⏳ 等待 DAG 結構出現...")
graph_loaded = False
for i in range(60):
    try:
        ready = driver.execute_script(
            "return !!window.__graph && !!window.__graph.nodes && Object.keys(window.__graph.nodes).length > 0"
        )
        if ready:
            print(f"✅ DAG 已載入完成（{i+1} 秒）")
            graph_loaded = True
            break
    except:
        pass
    time.sleep(1)

if not graph_loaded:
    print("❌ 超過 60 秒仍無法取得 DAG 資料，請確認網頁是否成功載入")
else:
    # 5. 擷取 DAG 節點與邊資料
    js_code = """
    let nodes = Object.values(window.__graph.nodes);
    let links = Object.values(window.__graph.links);
    return {
        nodes: nodes.map(n => ({id: n.id, status: n.status, x: n.x, y: n.y})),
        links: links.map(l => ({source: l.source, target: l.target}))
    };
    """
    dag_data = driver.execute_script(js_code)

    # 6. 儲存為 JSON 檔案
    with open("dag_snapshot.json", "w", encoding="utf-8") as f:
        json.dump(dag_data, f, ensure_ascii=False, indent=2)
    print("📁 已儲存 DAG 資料到 dag_snapshot.json，共有節點：", len(dag_data["nodes"]))

# 7. 結束（可選關閉瀏覽器）
# driver.quit()


✅ 已點擊暫停鍵
⏳ 等待 DAG 結構出現...
❌ 超過 60 秒仍無法取得 DAG 資料，請確認網頁是否成功載入
